In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import visualizer
import tag_util
import dataparser

In [ ]:
DF = pd.read_csv("ALL_PARSED.csv")
DF = dataparser.clean_df(DF, 3, False)

In [ ]:
# Load our tags
TAGS = pd.read_csv("TAGS.csv")

In [ ]:
TAGS

In [ ]:
TAGS['tags'].unique()

In [ ]:
tag_sets = TAGS.groupby("problem")["tags"].apply(set)
distinct_tag_sets = pd.Series(tag_sets.map(tuple).unique())
distinct_tag_sets

In [ ]:
def set_size(width, fraction=1):
    """Set figure dimensions to avoid scaling in LaTeX.

    Parameters
    ----------
    width: float
            Document textwidth or columnwidth in pts
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy

    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    # Width of figure (in pts)
    fig_width_pt = width * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    golden_ratio = (5**.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim

In [ ]:
width = 400

In [ ]:
# Filter rows where status is "Success"
solved_df = DF[DF['status'] == 'Success']
solved_df = solved_df[solved_df["sanity_sat"].isin(["sat", "unsat"])]  # Only accept sat or unsat as answer, some solvers might answer "unknown" or some error

In [ ]:
## Total time 
%matplotlib
_ = visualizer.sum_time_barchart(solved_df)

In [ ]:
## Total solved
%matplotlib
fig = visualizer.solved_barchart(DF, set_size(width))
fig.savefig("assets/total_solved_barchart.pdf", format="pdf", bbox_inches="tight")

In [ ]:
 print(visualizer.summary_table(DF).to_latex())

In [ ]:
visualizer.summary_table(tag_util.find_exact_tagset(DF, TAGS, set(["length_constraints", "substrings", "regular_constraints"])))

In [ ]:
%matplotlib
tagset = set(["regular_constraints"])
d = tag_util.find_exact_tagset(DF, TAGS, tagset)
fig = visualizer.solved_barchart(d, set_size(width))
fig.savefig(f"assets/{"_".join(tagset)}_solved_barchart.pdf", format="pdf", bbox_inches="tight")

In [ ]:
%matplotlib
tagset = set(["length_constraints", "regular_constraints", "substrings"])
d = tag_util.find_exact_tagset(DF, TAGS, tagset)
fig = visualizer.solved_barchart(d, set_size(width))
fig.savefig(f"assets/{"_".join(tagset)}_solved_barchart.pdf", format="pdf", bbox_inches="tight")

In [ ]:
%matplotlib
tagset = set(["lia", "length_constraints", "search", "substrings"])
d = tag_util.find_exact_tagset(DF, TAGS, tagset)
fig = visualizer.solved_barchart(d, set_size(width))
fig.savefig(f"assets/{"_".join(tagset)}_solved_barchart.pdf", format="pdf", bbox_inches="tight")

In [ ]:
%matplotlib
tagset = set(["re_replace", "length_constraints", "regular_constraints"])
d = tag_util.find_exact_tagset(DF, TAGS, tagset)
fig = visualizer.solved_barchart(d, set_size(width))
fig.savefig(f"assets/{"_".join(tagset)}_solved_barchart.pdf", format="pdf", bbox_inches="tight")

In [ ]:
%matplotlib
for tagset in distinct_tag_sets:
    df = tag_util.find_exact_tagset(DF, TAGS, set(tagset))
    if df.size > 0:
        fig = visualizer.sum_time_barchart(df)
        fig.suptitle(f'Tagset: {", ".join(tagset)}', fontsize=10, y=0.92, color='gray')
        

In [ ]:
%matplotlib inline
for tagset in distinct_tag_sets:
    df = tag_util.find_exact_tagset(DF, TAGS, set(tagset))
    
    if df.size > 0:
        fig = visualizer.solved_barchart(df)
        fig.suptitle(f'Tagset: {", ".join(tagset)}', fontsize=10, y=0.92, color='gray')

In [ ]:
import math
# Determine grid size based on the number of tagsets
num_tagsets = len(distinct_tag_sets)
cols = 3  # Number of columns (adjust as needed)
rows = math.ceil(num_tagsets / cols)  # Calculate the required number of rows

# Create a figure with subplots
fig, axes = plt.subplots(rows, cols, figsize=(5 * cols, 4 * rows))
axes = axes.flatten()  # Flatten in case of multiple rows and columns

for idx, tagset in enumerate(distinct_tag_sets):
    df = tag_util.find_exact_tagset(DF, TAGS, set(tagset))
    
    if df.size > 0:
        ax = axes[idx]
        visualizer.solved_barchart(df, ax=ax)  # Pass the subplot axis
        ax.set_title(f'Tagset: {", ".join(tagset)}', fontsize=10, color='gray')
        ax.legend(loc='lower left', bbox_to_anchor=(0, 0))
    else:
        axes[idx].axis('off')  # Hide unused subplot if no data

# Hide any remaining empty subplots
for j in range(idx + 1, len(axes)):
    axes[j].axis('off')

# Adjust layout for better spacing
plt.tight_layout()

plt.savefig('assets/combined_solved_barcharts.pdf', format="pdf")

In [ ]:
solved_problems = DF[DF['sanity_sat'].isin(['sat', 'unsat'])]['problem'].unique()
unsolved_problems = DF[~DF['problem'].isin(solved_problems)]['problem'].unique().tolist()

In [ ]:
len(unsolved_problems)

In [ ]:
unsolved_tags = TAGS[TAGS['problem'].isin(unsolved_problems)]

# Group by 'problem' and aggregate tags into sets
unsolved_tagsets = unsolved_tags.groupby('problem')['tags'].apply(set).reset_index()

# If you prefer a dictionary with problem as key and tagset as value:
unsolved_tagsets_dict = unsolved_tagsets.set_index('problem')['tags'].to_dict()

In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
unsolved_tagsets

In [ ]:
unsolved_tagsets['tagset'] = unsolved_tagsets['tags'].apply(frozenset)

# Now group by the tagset and count the number of problems
tagset_counts = unsolved_tagsets.groupby('tagset')['problem'].count().reset_index(name='count')


In [ ]:
tagset_counts

In [ ]:
print(tagset_counts.to_latex())

In [ ]:
mask = DF.groupby("problem")["sanity_sat"].transform(lambda x: all(v in {"sat", "unsat"} for v in x))
filtered_df = DF[~mask]
filtered_df = filtered_df[filtered_df["sanity_sat"].isin(["sat", "unsat"])]  # Only accept sat or unsat

In [ ]:
# Group by 'solver' and 'sanity_sat' and count the number of successes
sanity_counts = filtered_df.groupby(['solver', 'sanity_sat']).size().unstack(fill_value=0)

# Sort by total solved count (sum of 'sat' and 'unsat')
sanity_counts = sanity_counts.loc[sanity_counts.sum(axis=1).sort_values(ascending=False).index]

fig, ax = plt.subplots(figsize=set_size(width))


color_map = {'sat': 'green', 'unsat': 'orange'}
colors = [color_map.get(col, 'blue') for col in sanity_counts.columns]

# Plot the stacked bar chart with the sorted order
sanity_counts.plot(kind='bar', stacked=True, ax=ax, color=colors)

# Customize the plot
ax.set_xlabel('Solver')
ax.set_ylabel('Number of Problems Solved')

fig.savefig(f"assets/non_trivial_solved_barchart.pdf", format="pdf", bbox_inches="tight")